<a href="https://colab.research.google.com/github/teng-ny/Kaggle/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LightGBM 설치

In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM

fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [2]:
%cd /content/LightGBM

/content/LightGBM


In [3]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [4]:
!cmake -DUSE_GPU=1

-- Found OpenMP_C: -fopenmp  
-- Found OpenMP_CXX: -fopenmp  
-- Found OpenMP: TRUE   
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   filesystem
--   system
-- Using _mm_prefetch
-- Using _mm_malloc
-- Configuring done
-- Generating done
-- Build files have been written to: /content/LightGBM


In [5]:
!make -j$(nproc)

[  2%] Built target lightgbm_capi_objs
[ 89%] Built target lightgbm_objs
[ 97%] Built target lightgbm
[100%] Built target _lightgbm


In [6]:
!sudo apt-get -y install python-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.5).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cu

In [7]:
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [8]:
%cd /content/LightGBM/python-package/

/content/LightGBM/python-package


In [9]:
!sudo python setup.py install --precompile

running install
/usr/local/lib/python3.7/dist-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running build
running build_py
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
copying lightgbm/VERSION.txt -> build/lib/lightgbm
running install_lib
copying build/lib/lightgbm/VERSION.txt -> /usr/lib/python3.7/site-packages/lightgbm
INFO:LightGBM:Installing lib_lightgbm from: ['/content/LightGBM/lib_lightgbm.so']
running install_egg_info
removing '/usr/lib/python3.7/site-packages/lightgbm-3.3.2.99-py3.7.egg-info' 

In [10]:
!nvidia-smi

Tue Feb 22 06:50:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## lightGBM모델을 사용하기 위한 데이터 처리
### 각각의 트윗에서
- 반복되는 알파벳 제거
- 예수님, 부처님, 알라신의 이름은 god로 통일
- https / http / ftp 주소 제거
- \n \t 문자 제거
- 숫자 제거
- 두글자 이상의 단어만 남기고 제거
- 불용어 제거
- porterstemmer로 stemming
- 특수문자 제거 

In [11]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import pandas as pd

In [13]:
train=pd.read_csv("/content/train.csv")
test=pd.read_csv("/content/train.csv")

In [14]:
import re

alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
god_list = ['buddha', 'allah', 'jesus']
train_text_list=list(train['text'])
clear_text_list = []

In [15]:
for text in train_text_list:
  text_list_corpus = text.lower()
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
  clear_text=re.sub(pattern=pattern, repl=' ', string=text_list_corpus)
  clear_text=clear_text.replace('\n', ' ').replace('\t', ' ')

  clear_text=re.sub('[0-9]', ' ',clear_text)
  for i in range(len(alphabets)):
    clear_text=re.sub(alphabets[i]+'{3,}', alphabets[i], clear_text)
  for i in range(len(god_list)):
    clear_text=clear_text.replace(god_list[i], 'god')
  word_list=word_tokenize(clear_text)
  word_list=[word for word in word_list if len(word) > 2]
  word_list=[word for word in word_list if word not in stop_words]
  word_list=[stemmer.stem(word) for word in word_list]
  clear_text=' '.join(word_list)
  clear_text=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ', clear_text)
  clear_text_list.append(clear_text)
train['clear_text']=clear_text_list
train

,id,keyword,location,text,target,clear_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may god forgiv
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu sh...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridg collaps nearbi home
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,aria_ahrari thetawniest control wild fire cali...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utc volcano hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,polic investig e bik collid car littl portug e...


Bag of Words 만들기

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer=None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor=None, # 전처리 도구
                             stop_words=stop_words, # 불용어 nltk등의 도구를 사용할 수 있음
                             min_df=2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋음
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정함
                             max_features=1721 # 만들 피처의 수, 단어의 수
                             )
vectorizer

CountVectorizer(max_features=1721, min_df=2, ngram_range=(1, 3),
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [17]:
%%time
train_feature_vector=vectorizer.fit_transform(train['clear_text'])
train_feature_vector.shape

CPU times: user 946 ms, sys: 31.7 ms, total: 977 ms
Wall time: 1.44 s


In [18]:
vocab=vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

1721


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aba',
 'aba woman',
 'aba woman deliv',
 'abandon',
 'abc',
 'abc news',
 'ablaz',
 'absolut',
 'accid',
 'account']

In [19]:
import numpy as np

dist=np.sum(train_feature_vector, axis=0)

bog = pd.DataFrame(dist, columns=vocab)
bog

,aba,aba woman,aba woman deliv,abandon,abc,abc news,ablaz,absolut,accid,account,across,act,action,activ,actual,ad,ad video,ad video youtub,affect,affect fatal,affect fatal outbreak,africa,aftershock,ago,agre,ahead,air,air ambul,air ambul helicopt,aircraft,aircraft debri,aircraft debri found,airlin,airplan,airplan accid,airport,airport get,airport get swallow,alabama,alarm,...,wound suspect,wound suspect dead,wow,wreck,wreck wreck,wreckag,wreckag conclus,wreckag conclus confirm,write,wrong,wrought,wtf,ye,yeah,year,year ago,year atom,year atom bomb,year old,year old boy,yet,york,young,youth,youth save,youth save mani,youtub,youtub playlist,youtub video,zone,û_,ûª,ûªt,ûªv,ûï,ûïwhen,ûïwhen saw,ûïwhen saw coach,ûò,ûó
0,14,13,13,14,32,17,28,15,98,16,20,25,27,23,32,21,13,13,41,26,26,12,20,27,19,15,44,16,15,25,14,12,13,36,14,31,24,24,14,25,...,21,11,21,99,13,41,25,25,15,22,13,12,27,27,177,16,12,12,35,20,30,17,18,22,12,12,98,13,43,32,350,134,28,12,18,12,12,12,43,28


TF-IDF 가중치 적용하기

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer=TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(smooth_idf=False)

In [22]:
%%time
train_feature_tfidf=transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

CPU times: user 9.33 ms, sys: 940 µs, total: 10.3 ms
Wall time: 24.7 ms


lightGBM으로 학습하고 결과 제출하기

In [24]:
y_label=train['target']

In [31]:
import numpy as np
import pandas
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from subprocess import check_output

%matplotlib inline

In [34]:
from lightgbm import LGBMClassifier, plot_importance
lgb=LGBMClassifier()

제출

In [36]:
%time lgb. fit(train_feature_tfidf, y_label)

[LightGBM] [Info] Number of positive: 3271, number of negative: 4342
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.113685 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12524
[LightGBM] [Info] Number of data points in the train set: 7613, number of used features: 857
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.429660 -> initscore=-0.283239
[LightGBM] [Info] Start training from score -0.283239
CPU times: user 2.22 s, sys: 31 ms, total: 2.25 s
Wall time: 1.59 s


LGBMClassifier()

In [39]:
from scipy.stats.stats import Ttest_relResult
import re

alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
god_list = ['buddha', 'allah', 'jesus']
train_text_list=list(test['text'])
clear_text_list = []

for text in train_text_list:
  text_list_corpus = text.lower()
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
  clear_text=re.sub(pattern=pattern, repl=' ', string=text_list_corpus)
  clear_text=clear_text.replace('\n', ' ').replace('\t', ' ')

  clear_text=re.sub('[0-9]', ' ',clear_text)
  for i in range(len(alphabets)):
    clear_text=re.sub(alphabets[i]+'{3,}', alphabets[i], clear_text)
  for i in range(len(god_list)):
    clear_text=clear_text.replace(god_list[i], 'god')
  word_list=word_tokenize(clear_text)
  word_list=[word for word in word_list if len(word) > 2]
  word_list=[word for word in word_list if word not in stop_words]
  word_list=[stemmer.stem(word) for word in word_list]
  clear_text=' '.join(word_list)
  clear_text=re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ', clear_text)
  clear_text_list.append(clear_text)
test['clear_text']=clear_text_list
test

,id,keyword,location,text,target,clear_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may god forgiv
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu sh...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order california
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant crane hold bridg collaps nearbi home
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,aria_ahrari thetawniest control wild fire cali...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utc volcano hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,polic investig e bik collid car littl portug e...


In [41]:
%%time
test_vec=vectorizer.fit_transform(test['clear_text'])
test_vec.shape

CPU times: user 859 ms, sys: 17.2 ms, total: 876 ms
Wall time: 1.45 s


In [42]:
%%time
test_vec_tfidf=transformer.fit_transform(test_vec)
test_vec_tfidf.shape

CPU times: user 7.59 ms, sys: 31 µs, total: 7.62 ms
Wall time: 16.1 ms


In [44]:
pred2=lgb.predict_proba(test_vec_tfidf)
predict_labels=np.argmax(pred2, axis=1)
for i in range(len(predict_labels)):
  predict_labels[i] = predict_labels[i]

ids=list(test['id'])

submission_dic={'id':ids, 'target':predict_labels}
submission_df=pd.DataFrame(submission_dic)
submission_df.to_csv('kaggle_submission.csv', index=False)